### Connect to the database to create a basic version of the visual web page, the code is as follows: ###

In [ ]:
import pandas as pd
import pymysql as pm
import time
import geopandas as gpd
from shapely import geometry
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
import mapclassify as mc
import geoplot as gplt
import matplotlib
import matplotlib.patches as mpatches
import geoplot.crs as gcrs
import pyproj
from pyecharts.charts import Geo # Import geographic information processing module
from pyecharts import options as opts  # Configuration
from pyecharts.render import make_snapshot # Snapshot
from snapshot_selenium import snapshot 
from pyecharts.globals import ChartType, SymbolType # Global configuration

# Create connection object

conn = pm.connect(host = 'localhost' , port = 3306 , user = 'root' , password = '123456' , db = 'fever_clinic' , charset = 'utf8' ,
                  cursorclass = pm.cursors.DictCursor)
try:

# Obtain the cursor object
    
    with conn.cursor() as cursor:
        
# Execute SQL to get the result
        
        cursor.execute('select Hospital_Name,Hospital_Grade,Beds_Number,Longitude,Latitude from tb_nationalfeverclinic')
        data = cursor.fetchall()
        geo_sight_coord = {row['Hospital_Name'].strip(): [row['Longitude'] , row['Latitude']] for row in data} # Construct location dictionary data
        data_pair = []
        for row in data:
            if row['Hospital_Grade'] == 'Tertiary':
                data_pair.append((row['Hospital_Name'] , 3))
            elif row['Hospital_Grade'] == 'Secondary':
                data_pair.append((row['Hospital_Name'] , 2))
            elif row['Hospital_Grade'] == 'Primary':
                data_pair.append((row['Hospital_Name'] , 1))
            elif row['Hospital_Grade'] == 'Non-level':
                data_pair.append((row['Hospital_Name'] , 4))
except pm.MySQLError as error:
    print('Error！！')
    print(error)
finally:

# Close connection to release resources
    
    conn.close()

g=Geo(init_opts=opts.InitOpts(height='850px',width='1850px',bg_color='lightcyan')) # Geographical initialization
g.add_schema(maptype="china") # Limited scope
for key, value in geo_sight_coord.items(): # Cycle to geographic points
    g.add_coordinate(key, value[0], value[1]) # Additional point position

g.add("", data_pair, symbol_size=4) 
g.set_series_opts(label_opts=opts.LabelOpts(is_show=False), type='scatter')  # Scatter plot

fenlei = [
        {'min': 0, 'max': 1.5, 'label': 'Primary', 'color': '#00B2EE'},
        {'min': 1.5, 'max': 2.5, 'label': 'Secondary', 'color': '#71C671'},
        {'min': 2.5, 'max': 3.5, 'label': 'Tertiary', 'color': '#CD4F39'},
        {'min': 3.5, 'max': 4.5, 'label': 'Non-level','color': '#FF0000'} # There is a lower limit and no upper limit
         ]

g.set_global_opts(tooltip_opts=opts.TooltipOpts(is_show=False),
                visualmap_opts=opts.VisualMapOpts(is_show=True,is_piecewise=True,pieces=fenlei), 
                title_opts=opts.TitleOpts(title="Distribution of National Fever Clinics",
                                          pos_left=160,
                                          title_textstyle_opts=opts.TextStyleOpts(font_size=35,color='black'),
                                          subtitle="uploaded_time:2020-10-28",
                                          subtitle_textstyle_opts=opts.TextStyleOpts(font_size=20,color='whtie')
                                         )
                 )
make_snapshot(snapshot, g.render(r"E:\研究生\Fever Clinics Map.html"), 'Fever Clinics Map.png')

### Filter suspected duplicate data and save it locally, the code is as follows: ###

In [ ]:
import pandas as pd

# Read data

data = pd.read_excel(r'E:\研究生\疫情数据短评-Scientific data\National Fever Clinics.xlsx')

# Filter suspected duplicate data

data_may = []
latitude = data['Latitude']
longitude = data['Longitude']
latitude = list(latitude)
longitude = list(longitude)
location = list(zip(latitude , longitude))
for i in location:
    location.remove(i)
    for j in location:
        h = abs(i[0] - j[0])
        l = abs(i[1] - j[1])
        if h <= 0.0005 and l <= 0.0005:
            data_may.append(data[data['Latitude'] == i[0]])
            data_may.append(data[data['Latitude'] == j[0]])
df = data_may[0]
data_may.remove(df)
for i in data_may:
    df = df.append(i , ignore_index = True)

# Save data

df.to_excel(r'E:\研究生\suspect_duplicate_data.xlsx')

### Extract BD09 coordinates according to address information based on Baidu Map API, the code is as follows:

In [ ]:
import json
import requests as req
import sys
import os
import time
import xlwt
import pandas as pd
import socket

# Read data and create url

socket.setdefaulttimeout(20)
ty = sys.getfilesystemencoding()
ak = 'XokDzF9ylqjePtj4AalPmSxnmkCGeWrd' 
print(time.time())
print('start')
zuobiao = { 'Latitude' : [] , 'Longitude' : []}
urls = []
shujv = pd.read_excel('E:\研究生\Fever_Clinics.xlsx')
dizhi = shujv['Address']
chengshi = shujv['City']
for i in range (0,len(dizhi)):
    address = dizhi[i]
    city = chengshi[i]
    url = 'http://api.map.baidu.com/geocoding/v3/?address=' + str(address) + '&city=' + str(city) + '&output=json&ak=' + ak
    urls.append(url)
print('url list reading complete')

# Crawl BD09 coordinates

for url in urls:
    time.sleep(1) 
    html = req.get(url) # Read html
    data = html.json()  # Transform the format of html information into json format
    print(data)
    if data['status'] == 0:
        item = data['result']
        jlat = item['location']['lat']
        jlng = item['location']['lng']
        zuobiao['Latitude'].append(jlat)
        zuobiao['Longitude'].append(jlng)
    else:
        jlat = 'None'
        jlng = 'None'
        zuobiao['Latitude'].append(jlat)
        zuobiao['Longitude'].append(jlng)
    html.close()
print('Data Crawl Completed')

# Organize and save data

shujvbiao = pd.DataFrame(zuobiao)
shujvbiao.to_excel('E:\研究生\Coordinte_BD09.xlsx')
print('Data is saved locally')

### Transform the format of coordinate from BD09 to WGS84，the code is as follows:

In [ ]:
import pandas as pd
import math
from xlutils.copy import copy

# Read and process data

data = pd.read_excel('E:\研究生\Fever_Clinic.xlsx')
jingdu = data['Longitude']
weidu = data['Latitude']

# setup transform function

def BD09_to_WGS84(bd_lng , bd_lat):
    a = 6378245.0
    ee = 0.00669342162296594323
    PI = math.pi
    x_pi = 3.14159265358979324 * 3000.0 / 180.0
    x = bd_lng - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y , x) - 0.000003 * math.cos(x * x_pi)
    gcj_lng = z * math.cos(theta)
    gcj_lat = z * math.sin(theta)
    lat = gcj_lat - 35.0
    lng = gcj_lng - 105.0
    dlat = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
              0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    dlat += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
                math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    dlat += (20.0 * math.sin(lat * PI) + 40.0 *
                math.sin(lat / 3.0 * PI)) * 2.0 / 3.0
    dlat += (160.0 * math.sin(lat / 12.0 * PI) + 320 *
                math.sin(lat * PI / 30.0)) * 2.0 / 3.0
    dlng = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
              0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    dlng += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
                math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    dlng += (20.0 * math.sin(lng * PI) + 40.0 *
                math.sin(lng / 3.0 * PI)) * 2.0 / 3.0
    dlng += (150.0 * math.sin(lng / 12.0 * PI) + 300.0 *
                math.sin(lng / 30.0 * PI)) * 2.0 / 3.0
    radlat = gcj_lat / 180 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = gci_lat + dlat
    mglng = gcj_lng + dlng
    wgslng = gcj_lng * 2 - mglng
    wgslat = gci_lat * 2 - mglat
    return wgslng , wgslat

# Perform conversion operations

WGS = {'Longitude_WGS84' : [],
       'Latitude_WGS84' : []}
for i in range(0 , len(jingdu)):
    ma = BD09_to_WGS84(jingdu[i] , weidu[i])
    WGS['Longitude_WGS84'].append(ma[0])
    WGS['Latitude_WGS84'].append(ma[1])

# Organize and save data    

k = pd.DataFrame(WGS)
k.to_excel('E:\研究生\Coordinte_WGS84.xlsx')
print("Conversion complete")